# Setting

Mount your Google Drive

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set up mount symlink

In [153]:
# DRIVE_PATH = '/content/gdrive/My\ Drive/PointNet
DRIVE_PATH = '/content/gdrive/MyDrive/Berkeley/Coursework/2023Spring/CS282/Project-cs282'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/PointNet'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

## Import and others

In [154]:
import math
import random
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import time

In [155]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


## Download data
The url may change

In [158]:
%cd $SYM_PATH

# This may take a few minutes
if not os.path.exists("PointCloudAlchemist"):
    !git clone https://github.com/Joohwan-Seo/CS282A_final_project.git PointCloudAlchemist
else:
    print("Already downloaded.")

%cd PointCloudAlchemist

/content/gdrive/MyDrive/Berkeley/Coursework/2023Spring/CS282/Project-cs282
Cloning into 'PointCloudAlchemist'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Unpacking objects: 100% (23/23), 232.02 MiB | 8.05 MiB/s, done.
Updating files: 100% (10/10), done.
/content/gdrive/MyDrive/Berkeley/Coursework/2023Spring/CS282/Project-cs282/PointCloudAlchemist


In [ ]:
!pwd

/content/gdrive/MyDrive/Berkeley/Coursework/2023Spring/CS282/Project-cs282/PointCloudAlchemist


In [159]:
from utils import PointCloudData, Normalize, RotateXYZ, AddGaussianNoise, read_off
from network_sol import PointNetClassification, PointNetSegmentation, GetModel, PointNetLoss
from train_sol import TrainModel

## visualize image

In [ ]:
# v, f = read_off(DATA_PATH/"sofa/train/sofa_0001.off")
# print(v[0])
# print(f[0], type(f[0][0]))

In [ ]:
# i,j,k = np.array(f).T
# x,y,z = np.array(v).T
# print(len(x))

## Training
### Transformer for training.
1024 points per cloud as in the paper!

In [180]:
pointnet_tf = transforms.Compose([Normalize(),
                                  RotateXYZ(),
                                  AddGaussianNoise(),
                                  transforms.ToTensor()
                                ])
torch.manual_seed(340420)

all_data = {'train': PointCloudData(data_split = "train", method = 'random', acc_type = 'valid' , transform = pointnet_tf),
            'test' : PointCloudData(data_split = "test",  method = 'random', acc_type = 'valid' , transform = pointnet_tf)
            } 

all_dataloader = {'train': DataLoader(dataset = all_data['train'], batch_size = 32, shuffle = True),
                  'test' : DataLoader(dataset = all_data['test'] , batch_size = 64)
                 }

In [181]:
num_to_class = {i: cat for cat, i in all_data['train'].classes.items()}

for t in ['train', 'test']:
    print(f"# of {t} data = {len(all_data[t])}")

print(f"# of classes: {len(all_data['train'].classes)}")

any_idx = 3990
assert any_idx < len(all_data['train'])

print(f"Sampled pointcloud shape: {all_data['train'][any_idx]['pointcloud'].size()}")
print(f"Class of the {any_idx}-th train data = {num_to_class[all_data['train'][any_idx]['category']]}")

# of train data = 3991
# of test data = 908
# of classes: 10
Sampled pointcloud shape: torch.Size([1024, 3])
Class of the 3990-th train data = toilet


### Model
Three classes (`Tnet`, `Transform`, and `PointNet`) are defined above (model.py).  
We also need a loss function.

### Training loop

You can find a pretrained model here



In [186]:
# If you rerun this cell, you will lost all your work
model_dict, output_dict = {}, {}
for tnet in ['yes_t', 'no_t']:
    model_dict[tnet], output_dict[tnet] = {}, {}
    for task in ['Classification', 'Segmentation']:
        model_dict[tnet][task], output_dict[tnet][task] = None, None

In [183]:
tnet = 'yes_t' 
task = 'Classification'

# set model and check device
model_dict[tnet][task], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task].parameters()).device)

# train
output_dict[tnet][task] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

cuda:0
Epoch: 1  / 30, Batch: 25  / 125 >> Train loss: 1.987. Forward-backward path (s) = 0.08. Epoch time (s) = 2.39
Epoch: 1  / 30, Batch: 50  / 125 >> Train loss: 1.838. Forward-backward path (s) = 0.07. Epoch time (s) = 4.58
Epoch: 1  / 30, Batch: 75  / 125 >> Train loss: 1.792. Forward-backward path (s) = 0.08. Epoch time (s) = 6.77
Epoch: 1  / 30, Batch: 100 / 125 >> Train loss: 1.793. Forward-backward path (s) = 0.08. Epoch time (s) = 8.97
Epoch: 1  / 30, Batch: 125 / 125 >> Train loss: 1.679. Forward-backward path (s) = 0.05. Epoch time (s) = 11.15
Accuracy: 28%
---------------


KeyboardInterrupt: ignored

In [165]:
tnet = 'no_t' 
task = 'Classification'

# set model and check device
model_dict[tnet][task], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task].parameters()).device)

# train
output_dict[tnet][task] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

cuda:0
Epoch: 1  / 30, Batch: 25  / 125 >> Train loss: 1.429. Forward-backward path (s) = 0.02. Epoch time (s) = 0.95
Epoch: 1  / 30, Batch: 50  / 125 >> Train loss: 1.587. Forward-backward path (s) = 0.02. Epoch time (s) = 1.84
Epoch: 1  / 30, Batch: 75  / 125 >> Train loss: 1.22 . Forward-backward path (s) = 0.02. Epoch time (s) = 2.74
Epoch: 1  / 30, Batch: 100 / 125 >> Train loss: 1.451. Forward-backward path (s) = 0.02. Epoch time (s) = 3.75
Epoch: 1  / 30, Batch: 125 / 125 >> Train loss: 1.195. Forward-backward path (s) = 0.02. Epoch time (s) = 4.79
Accuracy: 48%
---------------
Epoch: 2  / 30, Batch: 25  / 125 >> Train loss: 1.473. Forward-backward path (s) = 0.02. Epoch time (s) = 1.04


KeyboardInterrupt: ignored

In [187]:
all_data = {'train': PointCloudData(data_split = "train", method = 'random', task = 'Segmentation', acc_type = 'valid' , transform = pointnet_tf),
            'test' : PointCloudData(data_split = "test",  method = 'random', task = 'Segmentation', acc_type = 'valid' , transform = pointnet_tf)
            } 
all_dataloader = {'train': DataLoader(dataset = all_data['train'], batch_size = 32, shuffle = True),
                  'test' : DataLoader(dataset = all_data['test'] , batch_size = 64)
                 }

In [188]:
tnet = 'yes_t' 
task = 'Segmentation'

# set model and check device
model_dict[tnet][task], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task].parameters()).device)

# train
output_dict[tnet][task] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

cuda:0
Epoch: 1  / 30, Batch: 25  / 45 >> Train loss: 1.215. Forward-backward path (s) = 0.26. Epoch time (s) = 7.54
Accuracy: 51%
---------------
Epoch: 2  / 30, Batch: 25  / 45 >> Train loss: 1.071. Forward-backward path (s) = 0.28. Epoch time (s) = 7.68
Accuracy: 61%
---------------


KeyboardInterrupt: ignored

In [189]:
tnet = 'no_t' 
task = 'Segmentation'

# set model and check device
model_dict[tnet][task], optimizer, scheduler = GetModel(tnet, task, DEVICE)
print(next(model_dict[tnet][task].parameters()).device)

# train
output_dict[tnet][task] = TrainModel(
    task         = task,
    model        = model_dict[tnet][task],
    train_loader = all_dataloader['train'], # training
    valid_loader = all_dataloader['test'],  # validation (and test)
    num_epochs   = 30,
    optimizer    = optimizer,
    scheduler    = scheduler,
    device       = DEVICE,
    save         = False
)

cuda:0
Epoch: 1  / 30, Batch: 25  / 45 >> Train loss: 1.309. Forward-backward path (s) = 0.17. Epoch time (s) = 4.94
Accuracy: 46%
---------------
Epoch: 2  / 30, Batch: 25  / 45 >> Train loss: 1.045. Forward-backward path (s) = 0.17. Epoch time (s) = 4.76
Accuracy: 38%
---------------


KeyboardInterrupt: ignored